<a href="https://colab.research.google.com/github/ggruszczynski/gpu_colab/blob/main/60_python_cuda_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python + cuda



In [3]:
# reference: https://numba.pydata.org/numba-doc/dev/cuda/reduction.html


from numba import cuda
from numba import jit
import numpy as np
from numba import vectorize, int32, int64, float32, float64
import matplotlib.pyplot as plt

%matplotlib inline

N = 4096
x = np.arange(N, dtype=np.float64) # [0...N] on the host
y = np.arange(N, dtype=np.float64)


d_x = cuda.to_device(x) # Copy of x on the device
d_y = cuda.to_device(y) # Copy of y on the device
d_out = cuda.device_array_like(d_x) # Like np.array_like, but for device arrays


SyntaxError: ignored

In [ ]:
@cuda.reduce
def sum_reduce(a, b):
    return a + b


expect = x.sum()      # numpy sum reduction
got = sum_reduce(x)   # cuda sum reduction
assert expect == got


In [ ]:
#Lambda functions can also be used here:
sum_reduce_lam = cuda.reduce(lambda a, b: a + b)

expect = x.sum()      # numpy sum reduction
got = sum_reduce_lam(x)   # cuda sum reduction
assert expect == got

In [ ]:
%timeit np.add(x, y)   # NumPy on CPU

In [ ]:
%timeit add_ufunc(x, y) # Numba on GPU

In [ ]:

@vectorize(['float64(float64, float64)'], target='cuda') # Type signature and target are required for the GPU
def add_ufunc(x, y):
    return x + y

## Reduction

In [ ]:
!nvidia-smi

Wed Feb 23 19:30:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## SAXPY

**SAXPY** stands for “Single-Precision A·X Plus Y”. It is a function in the standard Basic Linear Algebra Subroutines (BLAS)library.